In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_40/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


## Clasificacion: canjeo de coupones de descuento

Trabajaremos con el dataset preprocesado en la notebook 'opcional_preprocesamiento_datos_clase'.
El mismo cuenta con información de distintas campañas de marketing en donde se ofrecen coupnes de descuento sobre distintas marcas y productos. El dataset preprocesado cuenta con fetures con información sobre las campañas, sobre los consumidores (hábitos de consumo y características demográficas) y sobre los items alcanzados por las promociones.

La variable target es el estado de canje de los cupones: "redemption_status". 

El dataset original está [acá](https://www.kaggle.com/vasudeva009/coupon-redemption-smote-feature-selection/data). Se aconseja mirar la notebook de preprocesamiento para entender mejor las variables del dataset.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTENC
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif,RFECV,RFE
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,plot_confusion_matrix,roc_auc_score, classification_report, confusion_matrix, precision_recall_curve, auc

In [3]:
data=pd.read_csv('../Data/marketing/data_preprocessed.csv')
display(data.head(3))
data.dtypes

,Unnamed: 0,id,campaign_id,coupon_id,customer_id,redemption_status,campaign_type,duration,month,year,...,mean_selling_price_cust,mean_other_discount_cust,mean_coupon_discount_cust,mean_quantity_item_coupon,mean_selling_price_item_coupon,mean_other_discount_item_coupon,mean_coupon_discount_item_coupon,brand,brand_type,category
0,0,1,13,27,1053,0,X,47,5,2013,...,184.260484,-33.168935,-0.287258,1.169768,84.478583,-51.732305,-1.241694,1105,Established,Grocery
1,1,2,13,116,48,0,X,47,5,2013,...,234.247013,-27.699169,-3.215039,1.050926,63.833519,-10.661343,0.000000,56,Local,Grocery
2,2,6,9,635,205,0,Y,32,3,2013,...,121.094495,-17.795660,-2.212082,1.353802,332.128380,-142.498493,-2.987321,560,Established,Pharmaceutical


Unnamed: 0                            int64
id                                    int64
campaign_id                           int64
coupon_id                             int64
customer_id                           int64
redemption_status                     int64
campaign_type                        object
duration                              int64
month                                 int64
year                                  int64
age_range                            object
marital_status                       object
rented                                int64
family_size                          object
no_of_children                       object
income_bracket                        int64
mean_quantity_cust                  float64
mean_selling_price_cust             float64
mean_other_discount_cust            float64
mean_coupon_discount_cust           float64
mean_quantity_item_coupon           float64
mean_selling_price_item_coupon      float64
mean_other_discount_item_coupon 

In [4]:
data=data.drop(['Unnamed: 0','id','customer_id','campaign_id','coupon_id'],axis=1)

In [5]:
categorical=['campaign_type','month','year','age_range','marital_status','rented','family_size','no_of_children',\
             'income_bracket','brand','brand_type','category']

data[categorical]=data[categorical].astype('category')
data.dtypes

redemption_status                      int64
campaign_type                       category
duration                               int64
month                               category
year                                category
age_range                           category
marital_status                      category
rented                              category
family_size                         category
no_of_children                      category
income_bracket                      category
mean_quantity_cust                   float64
mean_selling_price_cust              float64
mean_other_discount_cust             float64
mean_coupon_discount_cust            float64
mean_quantity_item_coupon            float64
mean_selling_price_item_coupon       float64
mean_other_discount_item_coupon      float64
mean_coupon_discount_item_coupon     float64
brand                               category
brand_type                          category
category                            category
dtype: obj

### Primera parte
* Ver el balance de clases en el dataset conun value counts de la columna redemption_status
* Hacer un train-test-split estratificado
* Convertir las variables categoricas a dummy. Conviene que nos quedemos con una versión no dummy del dataset de entrenamiento puesto que luego usaremos SMOTENC
* Opcional (recomendado): Armar una función que tome como input un modelo entrenado y un dataset de testeo e imprima las métricas más importantes para evaluar clasificación (classification_report, matriz de confución, area bajo las curvas ROC y Precision-Recall)
* Instanciar un modelo Random Forest entrenarlo y evaluarlo en el dataset de testeo

In [6]:
data['redemption_status'].value_counts(normalize=True)

0    0.990698
1    0.009302
Name: redemption_status, dtype: float64

In [7]:
# Train test split

X_train,X_test,y_train,y_test=train_test_split(data.drop('redemption_status',axis=1),data['redemption_status'],\
                                               stratify=data['redemption_status'],random_state=0)


In [8]:
X_train_dummy=pd.get_dummies(X_train);
X_test_dummy=pd.get_dummies(X_test)
print('Mismas categorías en Train y Test:',(X_train_dummy.columns==X_test_dummy.columns).all())

Mismas categorías en Train y Test: True


In [9]:
def evaluate_model(model,X,y_true):
    '''
    Calcula las métricas ppales para evaluar un clasificador
    Toma como imput el modelo entrenado, el dataset de testeo y sus etiquetas
    '''
    y_pred=model.predict(X)
    y_proba=model.predict_proba(X)

    print(classification_report(y_true,y_pred))
    print('Area bajo la curva ROC:',np.round(roc_auc_score(y_true,y_proba[:,1]),4))
    precision, recall,threshold=precision_recall_curve(y_true,y_proba[:,1]);
    print('Area bajo la curva Precision-Recall:',np.round(auc(recall,precision),4))
    plot_confusion_matrix(model,X,y_true,cmap='Blues');
    return

In [10]:
model=RandomForestClassifier()
model.fit(X_train_dummy,y_train)
evaluate_model(model,X_test_dummy,y_test)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     19411
           1       0.50      0.01      0.02       182

    accuracy                           0.99     19593
   macro avg       0.75      0.51      0.51     19593
weighted avg       0.99      0.99      0.99     19593

Area bajo la curva ROC: 0.8702
Area bajo la curva Precision-Recall: 0.1332


### Resampling de las clases

* Hacer un undersampling de la clase mayoritaria en el training set para balancear las clases. ¿De qué tamaño quedó el dataset de entrenamiento?
* Volver a entrenar un random forest y evaluarlo en test
* Repetir el procedimiento usando SMOTENC para sobresamplear la clase minoritaria
* Instanciar un Random Forest usando class_weight='balanced_subsample', entrenarlo en el training set original (sin resampling) y evaluarlo en test set

* Combinar las tres estrategias anteriores:
- Undersampling
- Oversampling
- Class weight
Usar una combinación de undersampling y oversampling que les parezca razonable. Idealmente habría que optimizar esta combinación mediante cross-validation, pero no hay que subestimar el tiempo de cómputo. Utilicen una combinación de sampling_stategy que de por resultado un dataset de tamaño reducido respecto del original. Usaremos ese dataset para hacer feature selection en la segunda parte

In [11]:
# UnderSampling del dataset

sampler=RandomUnderSampler()
X_train_us,y_train_us=sampler.fit_resample(X_train_dummy,y_train)
print('X_train_us:',X_train_us.shape)
print('\nBalance de clases en train:')
print(y_train_us.value_counts())

print('\n\nX_test:',X_test_dummy.shape)
print('\nBalance de clases en test:')
print(y_test.value_counts())




X_train_us: (1094, 357)

Balance de clases en train:
1    547
0    547
Name: redemption_status, dtype: int64


X_test: (19593, 357)

Balance de clases en test:
0    19411
1      182
Name: redemption_status, dtype: int64


In [12]:
model=RandomForestClassifier()
model.fit(X_train_us,y_train_us)
print('DATASET SUBSAMPLEADO')
evaluate_model(model,X_test_dummy,y_test)

DATASET SUBSAMPLEADO
              precision    recall  f1-score   support

           0       1.00      0.83      0.90     19411
           1       0.04      0.85      0.08       182

    accuracy                           0.83     19593
   macro avg       0.52      0.84      0.49     19593
weighted avg       0.99      0.83      0.90     19593

Area bajo la curva ROC: 0.9152
Area bajo la curva Precision-Recall: 0.1316


In [13]:
print('DATASET SOBRESAMPLEADO')
categorical_mask=(X_train.dtypes=='category').values
sm=SMOTENC(categorical_features=categorical_mask,sampling_strategy='minority')
X_train_os,y_train_os=sm.fit_resample(X_train,y_train)
X_train_os=pd.get_dummies(X_train_os)

print('X_train_os:',X_train_os.shape)
print('\nBalance de clases en train:')
print(y_train_os.value_counts())

print('\n\nX_test:',X_test_dummy.shape)
print('\nBalance de clases en test:')
print(y_test.value_counts())

DATASET SOBRESAMPLEADO
X_train_os: (116458, 357)

Balance de clases en train:
1    58229
0    58229
Name: redemption_status, dtype: int64


X_test: (19593, 357)

Balance de clases en test:
0    19411
1      182
Name: redemption_status, dtype: int64


In [14]:
model=RandomForestClassifier()
model.fit(X_train_os,y_train_os)
evaluate_model(model,X_test_dummy,y_test)

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     19411
           1       0.24      0.14      0.17       182

    accuracy                           0.99     19593
   macro avg       0.62      0.57      0.58     19593
weighted avg       0.98      0.99      0.99     19593

Area bajo la curva ROC: 0.8725
Area bajo la curva Precision-Recall: 0.116


In [15]:
# Balance de clases
model=RandomForestClassifier(class_weight='balanced_subsample')
model.fit(X_train_dummy,y_train)
evaluate_model(model,X_test_dummy,y_test)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     19411
           1       0.57      0.02      0.04       182

    accuracy                           0.99     19593
   macro avg       0.78      0.51      0.52     19593
weighted avg       0.99      0.99      0.99     19593

Area bajo la curva ROC: 0.89
Area bajo la curva Precision-Recall: 0.173


In [16]:
# Combinando Estrategias
sampler=RandomUnderSampler(sampling_strategy=0.05)
X,y=sampler.fit_resample(X_train,y_train)
sm=SMOTENC(categorical_mask,sampling_strategy=0.2)
X_train_rs,y_train_rs=sm.fit_resample( X,y)

X_train_rs=pd.get_dummies(X_train_rs)
if not (X_test_dummy.columns==X_train_rs.columns).all():
    print('Train y Test tienen distintas Categorias:')
    print('Usar OneHotEncoding')

print(y_train_rs.shape)
print(y_train_rs.mean())
    


(13128,)
0.16666666666666666


In [17]:
# Balance de clases
model=RandomForestClassifier(class_weight='balanced_subsample')
model.fit(X_train_rs,y_train_rs)
evaluate_model(model,X_test_dummy,y_test)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     19411
           1       0.18      0.25      0.21       182

    accuracy                           0.98     19593
   macro avg       0.58      0.62      0.60     19593
weighted avg       0.99      0.98      0.98     19593

Area bajo la curva ROC: 0.9122
Area bajo la curva Precision-Recall: 0.129


### Segunda Parte: Selección de Features

#### SelectKBest

Primero seleccionemos featuers usando SelectKbest tomando como medida de score el criterio 'f_classif'

* Hacer un pipeline que concatene el selector de features y un RandomForestClassifier con class_weight='balanced_subsample'
* Hacer una gridsearchCV variando el parámetro k del selector de features. Usar scoring='f1'.
* Graficar los resultados del procedimiento de cross-validation: scores de clasificacion vs nro de features
* ¿Con cuántas features se quedarían? ¿Cuáles?
* ¿Cuál es el score en el test set al seleccionar el subset de features elegido?
* Graficar la importancia de las features para el modelo (atributo feature_importance_)

In [18]:
# Seleccion de features:
skf=StratifiedKFold(n_splits=3,shuffle=True,random_state=0)
steps=([('selector',SelectKBest(f_classif)),('classif',RandomForestClassifier(class_weight='balanced_subsample'))])
pipe=Pipeline(steps)
param_grid={'selector__k':np.arange(10,150,20)}
grid=GridSearchCV(pipe,param_grid,scoring='f1',cv=skf,verbose=3,n_jobs=3)
grid.fit(X_train_rs,y_train_rs)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  21 out of  21 | elapsed:   11.2s finished
/Users/csuarezgurruchaga/opt/anaconda3/envs/dhdsblend2021/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [315] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/Users/csuarezgurruchaga/opt/anaconda3/envs/dhdsblend2021/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('selector', SelectKBest()),
                                       ('classif',
                                        RandomForestClassifier(class_weight='balanced_subsample'))]),
             n_jobs=3,
             param_grid={'selector__k': array([ 10,  30,  50,  70,  90, 110, 130])},
             scoring='f1', verbose=3)

In [19]:
n_features=grid.cv_results_['param_selector__k'].data
mn_cv_score=grid.cv_results_['mean_test_score']
err=grid.cv_results_['std_test_score']
plt.bar(n_features,mn_cv_score,color = "r",width=3,yerr=err,align = "center")
plt.xlabel('Número de features')
plt.ylabel('test score');

In [20]:
# Evaluamos en test
skb=SelectKBest(f_classif,k=70)
X_train_reduced=skb.fit_transform(X_train_rs,y_train_rs)
X_test_reduced=X_test_dummy.loc[:,skb.get_support()]
model=RandomForestClassifier(class_weight='balanced_subsample')
model.fit(X_train_reduced,y_train_rs)
evaluate_model(model,X_test_reduced,y_test)

/Users/csuarezgurruchaga/opt/anaconda3/envs/dhdsblend2021/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [315] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
/Users/csuarezgurruchaga/opt/anaconda3/envs/dhdsblend2021/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


              precision    recall  f1-score   support

           0       0.99      0.99      0.99     19411
           1       0.16      0.28      0.21       182

    accuracy                           0.98     19593
   macro avg       0.58      0.63      0.60     19593
weighted avg       0.99      0.98      0.98     19593

Area bajo la curva ROC: 0.9021
Area bajo la curva Precision-Recall: 0.115


In [21]:
y=np.sort(model.feature_importances_)
x=np.argsort(model.feature_importances_)
x=x[::-1]
feat_names=X_train_rs.columns[skb.get_support()]
labels=feat_names[x]
y=y[::-1]

plt.figure(figsize=(15,8))
plt.bar(range(len(y)),y,color = "r",width=3,align = "center")
plt.xticks(range(len(y)), labels, rotation=90)

plt.xlim([0,30])

(0.0, 30.0)

#### Seleccion de Features por RFE
* Implementar RFECV usando como modelo un randomforest con clases pesadas. Usar un step grande (50) para tener una primera aproximacion
* Graficar los scores obtenidos para cada numero de features
* Usar RFE para seleccionar el número de features más adecuado. En este caso implementar un paso más fino (step=10), entrenando en todo el dataset de entremiento (sin cross-validation)
* Evaluar el modelo en el test set y mirar las feature_importance_

In [22]:
# Recursive feature elimination
skf=StratifiedKFold(n_splits=3,shuffle=True,random_state=0)
rfecv = RFECV(RandomForestClassifier(class_weight='balanced_subsample'), step = 50, cv=skf, scoring = 'f1', verbose=2)
rfecv.fit(X_train_rs,y_train_rs)

Fitting estimator with 357 features.
Fitting estimator with 307 features.
Fitting estimator with 257 features.
Fitting estimator with 207 features.
Fitting estimator with 157 features.
Fitting estimator with 107 features.
Fitting estimator with 57 features.
Fitting estimator with 7 features.
Fitting estimator with 357 features.
Fitting estimator with 307 features.
Fitting estimator with 257 features.
Fitting estimator with 207 features.
Fitting estimator with 157 features.
Fitting estimator with 107 features.
Fitting estimator with 57 features.
Fitting estimator with 7 features.
Fitting estimator with 357 features.
Fitting estimator with 307 features.
Fitting estimator with 257 features.
Fitting estimator with 207 features.
Fitting estimator with 157 features.
Fitting estimator with 107 features.
Fitting estimator with 57 features.
Fitting estimator with 7 features.


RFECV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
      estimator=RandomForestClassifier(class_weight='balanced_subsample'),
      scoring='f1', step=50, verbose=2)

In [23]:
#n_features=grid.cv_results_['param_selector__k'].data
mn_cv_score=rfecv.grid_scores_
n_features=np.arange(7,357+50,50)
n_features=np.insert(n_features,0,1)

plt.bar(n_features,mn_cv_score,color = "r",width=3,align = "center")
plt.xlabel('Número de features');
plt.ylabel('test score');

In [24]:
rfe=RFE(RandomForestClassifier(class_weight='balanced_subsample'),\
       n_features_to_select=70,step=10,verbose=1)

In [25]:
rfe.fit(X_train_rs,y_train_rs)

Fitting estimator with 357 features.
Fitting estimator with 347 features.
Fitting estimator with 337 features.
Fitting estimator with 327 features.
Fitting estimator with 317 features.
Fitting estimator with 307 features.
Fitting estimator with 297 features.
Fitting estimator with 287 features.
Fitting estimator with 277 features.
Fitting estimator with 267 features.
Fitting estimator with 257 features.
Fitting estimator with 247 features.
Fitting estimator with 237 features.
Fitting estimator with 227 features.
Fitting estimator with 217 features.
Fitting estimator with 207 features.
Fitting estimator with 197 features.
Fitting estimator with 187 features.
Fitting estimator with 177 features.
Fitting estimator with 167 features.
Fitting estimator with 157 features.
Fitting estimator with 147 features.
Fitting estimator with 137 features.
Fitting estimator with 127 features.
Fitting estimator with 117 features.
Fitting estimator with 107 features.
Fitting estimator with 97 features.
Fi

RFE(estimator=RandomForestClassifier(class_weight='balanced_subsample'),
    n_features_to_select=70, step=10, verbose=1)

In [26]:
X_train_reduced=X_train_rs.loc[:,rfe.support_]
X_test_reduced=X_test_dummy.loc[:,rfe.support_]
model=RandomForestClassifier(class_weight='balanced_subsample')
model.fit(X_train_reduced,y_train_rs)
evaluate_model(model,X_test_reduced,y_test)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     19411
           1       0.19      0.29      0.23       182

    accuracy                           0.98     19593
   macro avg       0.59      0.64      0.61     19593
weighted avg       0.99      0.98      0.98     19593

Area bajo la curva ROC: 0.9134
Area bajo la curva Precision-Recall: 0.131


In [27]:
y=np.sort(model.feature_importances_)
x=np.argsort(model.feature_importances_)
x=x[::-1]
feat_names=X_train_rs.columns[skb.get_support()]
labels=feat_names[x]
y=y[::-1]

plt.figure(figsize=(15,8))
plt.bar(range(len(y)),y,color = "r",width=3,align = "center")
plt.xticks(range(len(y)),labels,rotation=90);
plt.xlim([0,30])

(0.0, 30.0)